1. Total number of donations made on mobile devices in 2021
2. Average donation amount made through mobile devices in 2021
3. Conversion rate for mobile devices in 2021
4. The percentage of donors that made a donation or completed a registration through a mobile device in response to an email in 2021

_\#4 is not possible to calculate since we can not presently tie a transaction to an email_

In [1]:
import sys, datetime
sys.path.append("../../scripts/")
from s3_support import *

import pandas as pd
import numpy as np

# 1. total number of donations made on mobile devices in 2021

In [16]:
q = '''select 
            source,
            count(id), 
            count(distinct(id)) as distinct_ids,
            sum(amount) 
        from transactions 
        where 
            status='A' and
            (source='mobile' or source='sms') and
            year=2021
        group by source'''
df = redshift_query_read(q, schema='production')

In [18]:
for source in df['source'].tolist():
    print(source)
    print("{:,} transactions".format(df[df['source']==source]['count'].iloc[0]))
    print("${:,} dollars".format(df[df['source']==source]['sum'].iloc[0]))
    print()
print("all")
print("{:,} transactions".format(df['count'].sum()))
print("${:,} dollars".format(df['sum'].sum()))

mobile
295,935 transactions
$37,965,726.0000001 dollars

sms
19,457 transactions
$2,289,211.67 dollars

all
315,392 transactions
$40,254,937.6700001 dollars


# 2. Average donation amount made through mobile devices in 2021

In [19]:
q = '''select 
            source, 
            amount 
        from transactions
        where 
            status='A' and
            (source='mobile' or source='sms') and
            year=2021'''
df = redshift_query_read(q, schema='production')

In [20]:
df.groupby('source')['amount'].agg(['mean', 'median']).reset_index()

,source,mean,median
0,mobile,128.290760,50.0
1,sms,117.654914,35.0


In [21]:
df['amount'].agg(['mean', 'median'])

mean      127.634619
median     50.000000
Name: amount, dtype: float64

# 3. Conversion rate for mobile devices in 2021

In [22]:
# load transactions with device data
q = '''select
            id,
            amount,
            source,
            useragent
        from transactions
        where 
            status='A' and
            (source='mobile' or source='sms') and
            year=2021'''
trans = redshift_query_read(q, schema='production')

In [39]:
trans['is_tablet'] = trans['useragent'].str.contains('iPad')|trans['useragent'].str.contains('tablet')

In [40]:
trans['is_tablet'].value_counts()

False    315312
True         80
Name: is_tablet, dtype: int64

In [44]:
# load traffic
q = '''select
            views,
            devicecategory,
            case path like '%sms%'
                when true then 1
                else 0
            end as is_sms
        from googleanalytics_traffic
        where
            (devicecategory='mobile' or devicecategory='tablet') and
            qgiv_frontend='true' and
            date_part('year', date)=2021'''
traf = redshift_query_read(q, schema='production')

In [35]:
traf.groupby('devicecategory')['views'].sum()

devicecategory
mobile    7469102
tablet     457494
Name: views, dtype: int64

In [41]:
trans.groupby('source')['id'].count()

source
mobile    295935
sms        19457
Name: id, dtype: int64

In [43]:
all_conv = trans['id'].count() / traf['views'].sum()
non_tablet_conv = trans[~trans['is_tablet']]['id'].count() / traf[traf['devicecategory']=='mobile']['views'].sum()

print("Overall conversion: {:.1f}%".format(all_conv * 100))
print("Removing tablets: {:.1f}%".format(non_tablet_conv * 100))

Overall conversion: 4.0%
Removing tablets: 4.2%


In [46]:
conv_mobile = trans[trans['source']=='mobile']['id'].count() / traf[traf['is_sms']==0]['views'].sum()
conv_sms = trans[trans['source']=='sms']['id'].count() / traf[traf['is_sms']==1]['views'].sum()

print("Isolated mobile: {:.1f}%".format(conv_mobile * 100))
print("Isolated sms: {:.1f}%".format(conv_sms * 100))

Isolated mobile: 3.7%
Isolated sms: 73.3%


# 1. Total number of donations made on mobile devices in 2021

Queried by accepted transactions by source ('mobile' & 'sms')

| Source |  Total  |
|--------|---------|
| Mobile | 295,935 |
| SMS    |  19,457 |
| All    | 315,392 |

# 2. Average donation amount made through mobile devices in 2021

| Source |   Mean  | Median |
|--------|---------|--------|
| Mobile | $128.29 | $50.00 |
| SMS    | $117.65 | $35.00 |
| All    | $127.63 | $50.00 |

# 3. Conversion rate for mobile devices in 2021

| Group            | Conversion |
|------------------|------------|
| Overall          |   4.0%     |
| Removing tablets |   4.2%     |
| Isolated mobile  |   3.7%     |
| Isolated SMS     |  73.3%     |

_As noted previously, identification of transactions originating from Android tablets is not reliable so the "Removing tablets" value is not to be relied upon beyond removing iPads._

# 4. The percentage of donors that made a donation or completed a registration through a mobile device in response to an email in 2021

_We can not presently tie a transaction to an email_